In [1]:
import subprocess
import rioxarray 
import xarray as xr
import numpy as np

> This notebook needs to be done only once if you don't have the cleaned paleo-elevation already...

There are some weird elevations in the `Pacific` ocean (mainly close to the edges of Scotese paleo-elevation model) as shown in the figure below (left hand side is the Scotese elevation at 95 Ma).

<img src="img/ndem.png"
     alt="Removing weird edge effects"
     style="float: left; margin-right: 10px;" />

In this notebook, I remove these edges problems and store the new files (right globe in the above figure) in a new folder.

I first define a function to remove intermediate netcdf files created during the cleaning processes...

In [2]:
def runSubProcess(args, output=False, cwd="."):
    '''
    This function takes an initial set of unix commands and execute them outside the Python interpreter.
    '''
    
    # Launch a subprocess
    p = subprocess.Popen(
        args,
        cwd=cwd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
    )

    # Capture and re-print OpenMC output in real-time
    lines = []
    while True:
        # If OpenMC is finished, break loop
        line = p.stdout.readline()
        if not line and p.poll() is not None:
            break

        lines.append(line)
        if output:
            # If user requested output, print to screen
            print(line, end="")

    # Raise an exception if return status is non-zero
    if p.returncode != 0:
        # Get error message from output and simplify whitespace
        output = "".join(lines)
        if "ERROR: " in output:
            _, _, error_msg = output.partition("ERROR: ")
        elif "what()" in output:
            _, _, error_msg = output.partition("what(): ")
        else:
            error_msg = "dbscan aborted unexpectedly."
        error_msg = " ".join(error_msg.split())

        raise RuntimeError(error_msg)

Set of unix command that will be performed (here it will be rm test.nc) 

In [3]:
# Basically we remove the file test.nc
args = ['rm','test.nc',]

Define the time we want to use, basically here `t` will be set so it varies from 100 to 0 (with 5 increments: 100, 95, 90, 85...).

> We could do a loop but I check the newly created file in Paraview to see if there are any issues and if we need to adjust the parameters...)

In [4]:
t = 0

# Open with xarray the Scotese paleo-elevation at the considered time step
data = xr.open_dataset('data/dem/'+str(t)+'Ma.nc')

elev = data.z.values.copy()

# Set edges to nan
elev[:,-10:] = np.nan 
elev[:10,:] = np.nan
elev[-10:,:] = np.nan


# Also set values above specific elevation to 0. 
# there are also in addition to the edges issues several places where 
# there are some singular peaks in the initial elevations. 
# Here we simply set those peaks to 0 (could potentially do something 
# better at some stage...)
if t > 40:
    elev[elev>8000] = 0
else:
    elev[elev>9000] = 0

# Store the updated elevation with a new name `nz` in the xarray dataset
data["nz"]=(['latitude', 'longitude'],  elev)
data = data.drop(['z'])
data['z'] = data['nz']
data = data.drop(['nz'])
data.to_netcdf('test.nc')
data.close()

# Add a coordinate reference system to the file to interpolate nan values
rds = rioxarray.open_rasterio('test.nc', mask_and_scale=False)
rds = rds.rio.write_crs(4326)
rds = rds.rio.interpolate_na()
rds.close()

# Clean the dataset and save it to a new file
data2 = xr.open_dataset('test.nc')
data2["nz"]=(['latitude', 'longitude'],  rds.values[0,:,:])
data2 = data2.drop(['z'])
data2['z'] = data2['nz']
data2 = data2.drop(['nz'])
data2.to_netcdf('data/ndem/'+str(t)+'Ma.nc')
data2.close()

# Remove the `test.nc` file
runSubProcess(args)

## 100 Ma

Adjustments have been done for 100Ma for example, where instead of the above I used:

In [ ]:
# Open it with xarray
data = xr.open_dataset('data/dem/100Ma.nc')
elev = data.z.values.copy()

elev[:,-1] = elev[:,0]
elev[elev>8000] = 0

data["nz"]=(['latitude', 'longitude'],  elev)
data = data.drop(['z'])
data['z'] = data['nz']
data = data.drop(['nz'])

data.to_netcdf('data/ndem/100Ma.nc')